# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'10-12-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'10-12-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-10-13 04:24:02,33.93911,67.709953,39870,1479,33118,5273.0,Afghanistan,102.418955,3.709556
1,NaN,NaN,NaN,Albania,2020-10-13 04:24:02,41.15330,20.168300,15570,424,9585,5561.0,Albania,541.038293,2.723186
2,NaN,NaN,NaN,Algeria,2020-10-13 04:24:02,28.03390,1.659600,53325,1809,37382,14034.0,Algeria,121.376817,3.392405
3,NaN,NaN,NaN,Andorra,2020-10-13 04:24:02,42.50630,1.521800,2995,57,1928,1010.0,Andorra,3876.269980,1.903172
4,NaN,NaN,NaN,Angola,2020-10-13 04:24:02,-11.20270,17.873900,6488,219,2744,3525.0,Angola,19.740605,3.375462


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,39297,39341,39422,39486,39548,39616,39693,39703,39799,39870
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,14117,14266,14410,14568,14730,14899,15066,15231,15399,15570
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,51995,52136,52270,52399,52520,52658,52804,52940,53072,53325


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1462,1462,1466,1467,1469,1470,1472,1473,1477,1479
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,392,396,400,403,407,411,413,416,420,424
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1756,1760,1768,1768,1771,1783,1789,1795,1801,1809


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,32842,32852,32879,32977,33045,33058,33058,33064,33114,33118
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,8536,8675,8825,8965,9115,9215,9304,9406,9500,9585
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,36482,36578,36672,36763,36857,36958,37067,37170,37170,37382


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1818,1828,1831,1839,1852,1863,1882,1898,1905,1911
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,6048,6073,6085,6116,6134,6141,6172,6190,6203,6220
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,921,921,921,923,927,927,939,942,942,944


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,27,27,27,27,27,28,28,28,28,28
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,53,53,53,55,56,64,64,65,65,65
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,9,9,9,9,9


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
633,1001.0,Autauga,Alabama,US,2020-10-13 04:24:02,32.539527,-86.644082,1911,28,0,1883.0,"Autauga, Alabama, US",3420.501530,1.465201
670,1075.0,Lamar,Alabama,US,2020-10-13 04:24:02,33.779950,-88.096680,423,3,0,420.0,"Lamar, Alabama, US",3064.107208,0.709220
671,1077.0,Lauderdale,Alabama,US,2020-10-13 04:24:02,34.901719,-87.656247,1935,37,0,1898.0,"Lauderdale, Alabama, US",2086.725835,1.912145


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,7804199,215085,3106728
India,7175880,109856,6227295
Brazil,5103408,150689,4526393


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,7804199,215085,3106728,4482386,2020-10-13 04:24:02,37.923092,-91.345036
India,7175880,109856,6227295,838729,2020-10-13 04:24:02,23.088275,81.806127
Brazil,5103408,150689,4526393,426326,2020-10-13 04:24:02,-12.669522,-48.480493
Russia,1305093,22594,1019905,262594,2020-10-13 04:24:02,54.546312,62.120860
Colombia,919084,27985,798396,92703,2020-10-13 04:24:02,5.349139,-74.190797


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,857913,16586,0
Texas,821110,16995,0
Florida,736024,15412,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,857913,16586,0,841327,2020-10-13 04:24:02,37.843962,-120.728594
Texas,821110,16995,0,804115,2020-10-13 04:24:02,31.660643,-98.653069
Florida,736024,15412,0,720612,2020-10-13 04:24:02,28.940755,-82.700744
New York,475315,33301,0,442014,2020-10-13 04:24:02,42.544151,-75.474183
Georgia,332311,7429,0,324882,2020-10-13 04:24:02,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,282982,6773,0
Florida,Miami-Dade,175397,3445,0
Illinois,Cook,155567,5304,0
Texas,Harris,151463,2682,0
Arizona,Maricopa,146574,3466,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,282982,6773,0,276209,2020-10-13 04:24:02,34.308284,-118.228241,6037.0
Florida,Miami-Dade,175397,3445,0,171952,2020-10-13 04:24:02,25.611236,-80.551706,12086.0
Illinois,Cook,155567,5304,0,150263,2020-10-13 04:24:02,41.841448,-87.816588,17031.0
Texas,Harris,151463,2682,0,148781,2020-10-13 04:24:02,29.858649,-95.393395,48201.0
Arizona,Maricopa,146574,3466,0,143108,2020-10-13 04:24:02,33.348359,-112.491815,4013.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-10,39703,15231,52940,2696,6246,111,883882,55736,27263,54423,...,593565,2268,60776,82453,1107,43945,10,2051,15415,8010
2020-10-11,39799,15399,53072,2696,6366,111,894206,56451,27285,55319,...,606447,2294,61098,83137,1109,44299,10,2052,15458,8011
2020-10-12,39870,15570,53325,2995,6488,111,903730,56821,27310,56298,...,620458,2313,61319,83756,1110,44684,10,2052,15549,8021


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-10,1473,416,1795,55,218,3,23581,1016,898,852,...,42850,50,502,691,35,378,1,595,337,230
2020-10-11,1477,420,1801,55,218,3,23868,1020,898,851,...,42915,50,505,697,35,381,1,595,337,230
2020-10-12,1479,424,1809,57,219,3,24186,1026,899,855,...,42965,51,509,704,35,387,1,596,345,230


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-10,33064,9406,37170,1814,2716,97,709464,45771,24987,42829,...,2495,1930,57704,73919,1024,37240,8,1329,14541,6492
2020-10-11,33114,9500,37170,1814,2743,97,721380,45824,25000,43448,...,2503,1942,58069,74664,1024,37942,8,1329,14599,6504
2020-10-12,33118,9585,37382,1928,2744,97,732582,45891,25029,44065,...,2511,1950,58306,75400,1025,38228,8,1329,14682,7627


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1818,1828,1831,1839,1852,1863,1882,1898,1905,1911
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,397,400,402,403,402,404,407,413,420,423
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1807,1815,1832,1851,1862,1880,1894,1914,1918,1935
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,652,652,662,676,685,689,699,711,716,729
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,6117,6141,6156,6173,6218,6243,6268,6281,6292,6325


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-10,164526,9431,0,224978,92220,852410,77618,60038,21998,49,...,212649,815915,84644,1857,1325,157609,93035,17927,147560,7455
2020-10-11,165342,9686,0,225575,92833,855190,78437,60038,22130,49,...,214717,818426,85844,1868,1325,158409,93035,18132,150236,7611
2020-10-12,166076,9877,0,226050,93487,857913,79013,61377,22289,49,...,217682,821110,86832,1876,1325,159248,93035,18281,152192,7802


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-10        1898    6190     942  726   1704     623    952    3823   
2020-10-11        1905    6203     942  736   1713     624    958    3841   
2020-10-12        1911    6220     944  738   1722     625    962    3861   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-10         1242      666  ...     310     58      400      131   
2020-10-11         1242      670  ...     326     61      408      133   
2020-10-12         1245      674  ...     339     65      428      133   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-10            396   658   379          0      124     58  
2020-10-11            394   662   380          0      125     58  
2020-10-12            394   675   381          0      126     58  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-10,2664,60,0,5759,1552,16567,2112,4530,653,0,...,2758,16943,510,58,20,3351,2190,387,1458,54
2020-10-11,2664,60,0,5759,1569,16579,2113,4530,654,0,...,2767,16971,517,58,20,3355,2190,385,1465,54
2020-10-12,2665,60,0,5759,1586,16586,2116,4532,656,0,...,2774,16995,522,58,20,3358,2190,388,1474,54


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-10          28      65       9   12     16      16     40      49   
2020-10-11          28      65       9   12     16      16     40      49   
2020-10-12          28      65       9   12     16      16     40      49   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-10           43       13  ...       2      1        4        1   
2020-10-11           43       13  ...       2      1        4        1   
2020-10-12           43       13  ...       2      1        4        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-10              2     1     2          1        6      0  
2020-10-11              2     1     2          1        6      0  
2020-10-12              2     1     2          0        6      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-10,0.000252,0.010952,0.002576,0.000000,0.035649,0.0,0.014245,0.011781,0.000697,0.023220,...,0.026237,0.007552,0.007192,0.009266,0.001810,0.006436,0.0,0.000000,0.004955,0.002002
2020-10-11,0.002418,0.011030,0.002493,0.000000,0.019212,0.0,0.011680,0.012828,0.000807,0.016464,...,0.021703,0.011464,0.005298,0.008296,0.001807,0.008056,0.0,0.000488,0.002789,0.000125
2020-10-12,0.001784,0.011105,0.004767,0.110905,0.019164,0.0,0.010651,0.006554,0.000916,0.017697,...,0.023103,0.008282,0.003617,0.007446,0.000902,0.008691,0.0,0.000000,0.005887,0.001248


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-10,0.000679,0.007264,0.003354,0.000000,0.028302,0.0,0.015328,0.005941,0.001115,0.011876,...,0.001894,0.020408,0.008032,0.010234,0.0,0.029973,0.0,0.003373,0.002976,0.004367
2020-10-11,0.002716,0.009615,0.003343,0.000000,0.000000,0.0,0.012171,0.003937,0.000000,-0.001174,...,0.001517,0.000000,0.005976,0.008683,0.0,0.007937,0.0,0.000000,0.000000,0.000000
2020-10-12,0.001354,0.009524,0.004442,0.036364,0.004587,0.0,0.013323,0.005882,0.001114,0.004700,...,0.001165,0.020000,0.007921,0.010043,0.0,0.015748,0.0,0.001681,0.023739,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-10,0.000181,0.010963,0.002779,0.000000,0.011546,0.0,0.017676,0.005337,0.00000,0.018792,...,0.002814,0.006781,0.008441,0.012312,0.000000,0.008613,0.0,0.0,0.007064,0.002780
2020-10-11,0.001512,0.009994,0.000000,0.000000,0.009941,0.0,0.016796,0.001158,0.00052,0.014453,...,0.003206,0.006218,0.006325,0.010079,0.000000,0.018851,0.0,0.0,0.003989,0.001848
2020-10-12,0.000121,0.008947,0.005704,0.062845,0.000365,0.0,0.015529,0.001462,0.00116,0.014201,...,0.003196,0.004119,0.004081,0.009857,0.000977,0.007538,0.0,0.0,0.005685,0.172663


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-10,0.006491,0.027118,NaN,0.003990,0.009944,0.003927,0.013356,0.000000,0.007834,0.0,...,0.007801,0.004585,0.016256,0.005959,0.000755,0.007911,0.005132,0.012424,0.018934,0.016360
2020-10-11,0.004960,0.027038,NaN,0.002654,0.006647,0.003261,0.010552,0.000000,0.006001,0.0,...,0.009725,0.003078,0.014177,0.005924,0.000000,0.005076,0.000000,0.011435,0.018135,0.020926
2020-10-12,0.004439,0.019719,NaN,0.002106,0.007045,0.003184,0.007343,0.022303,0.007185,0.0,...,0.013809,0.003279,0.011509,0.004283,0.000000,0.005296,0.000000,0.008218,0.013020,0.025095


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-10      0.008502  0.002916  0.003195  0.009736  0.008881  0.001608   
2020-10-11      0.003688  0.002100  0.000000  0.013774  0.005282  0.001605   
2020-10-12      0.003150  0.002741  0.002123  0.002717  0.005254  0.001603   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-10      0.009544  0.005259  0.003231  0.006042  ...  0.019737   
2020-10-11      0.006303  0.004708  0.000000  0.006006  ...  0.051613   
2020-10-12      0.004175  0.005207  0.002415  0.005970  ...  0.039877   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-10      0.000000  0.028278  0.007692   0.007634  0.003049  0.002646   
2020-10-11      0.051724  0.020000  0.015267  -0.005051  0.006079  0.002639   
2020-10-12      0.065574  0.049020  0.000000   0.000000  0.019637  0.002632   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-10           -1.0  0.008130  0.074074  
2020-10-11            NaN  0.008065  0.000000  
2020-10-12            NaN  0.008000  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-10,0.004146,0.0,NaN,0.002262,0.014379,0.004913,0.004280,0.000000,0.003072,NaN,...,0.009517,0.004268,0.009901,0.0,0.0,0.002993,0.0,0.013089,0.012500,0.0
2020-10-11,0.000000,0.0,NaN,0.000000,0.010954,0.000724,0.000473,0.000000,0.001531,NaN,...,0.003263,0.001653,0.013725,0.0,0.0,0.001194,0.0,-0.005168,0.004801,0.0
2020-10-12,0.000375,0.0,NaN,0.000000,0.010835,0.000422,0.001420,0.000442,0.003058,NaN,...,0.002530,0.001414,0.009671,0.0,0.0,0.000894,0.0,0.007792,0.006143,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2020-10-10         0.0  0.015625     0.0  0.0    0.0     0.0    0.0  0.042553   
2020-10-11         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   
2020-10-12         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-10          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2020-10-11          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2020-10-12          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-10            0.0   0.0   0.0        0.0      0.0    NaN  
2020-10-11            0.0   0.0   0.0        0.0      0.0    NaN  
2020-10-12            0.0   0.0   0.0       -1.0      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-10,0.001018,0.011076,0.002615,0.019696,0.027694,0.004192,0.016068,0.011629,0.000695,0.022408,...,0.026747,0.009121,0.006527,0.008723,0.002262,0.008069,4.279844e-35,0.000342,0.004247,0.002997
2020-10-11,0.001718,0.011053,0.002554,0.009848,0.023453,0.002096,0.013874,0.012229,0.000751,0.019436,...,0.024225,0.010292,0.005912,0.008509,0.002034,0.008062,2.139922e-35,0.000415,0.003518,0.001561
2020-10-12,0.001751,0.011079,0.003661,0.060376,0.021309,0.001048,0.012263,0.009391,0.000834,0.018567,...,0.023664,0.009287,0.004765,0.007977,0.001468,0.008376,1.069961e-35,0.000207,0.004703,0.001405


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-10,0.000926,0.007215,0.003592,0.006988,0.019622,1.315257e-54,0.018423,0.006290,0.000661,0.009272,...,0.001865,0.010860,0.007079,0.009781,1.446834e-13,0.024569,0.0,0.001799,0.002378,0.002321
2020-10-11,0.001821,0.008415,0.003467,0.003494,0.009811,6.576283e-55,0.015297,0.005113,0.000331,0.004049,...,0.001691,0.005430,0.006528,0.009232,7.234172e-14,0.016253,0.0,0.000899,0.001189,0.001161
2020-10-12,0.001587,0.008969,0.003955,0.019929,0.007199,3.288142e-55,0.014310,0.005498,0.000722,0.004375,...,0.001428,0.012715,0.007224,0.009638,3.617086e-14,0.016000,0.0,0.001290,0.012464,0.000580


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-10,0.000380,0.011405,0.002789,0.019275,0.013548,0.000777,0.018065,0.004884,0.000425,0.018540,...,0.005744,0.006951,0.008011,0.011697,0.000329,0.010447,2.507721e-37,0.000340,0.005383,0.002883
2020-10-11,0.000946,0.010699,0.001395,0.009638,0.011744,0.000388,0.017430,0.003021,0.000473,0.016496,...,0.004475,0.006584,0.007168,0.010888,0.000164,0.014649,1.253861e-37,0.000170,0.004686,0.002366
2020-10-12,0.000534,0.009823,0.003549,0.036241,0.006055,0.000194,0.016479,0.002241,0.000816,0.015349,...,0.003836,0.005352,0.005625,0.010373,0.000570,0.011093,6.269303e-38,0.000085,0.005186,0.087514


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-10,0.006651,0.022361,NaN,0.003592,0.010989,0.004079,0.011988,0.002462,0.006944,8.275286e-64,...,0.008151,0.005026,0.016109,0.005446,0.000541,0.007720,0.005958,0.014345,0.019573,0.022747
2020-10-11,0.005805,0.024700,NaN,0.003123,0.008818,0.003670,0.011270,0.001231,0.006472,4.137643e-64,...,0.008938,0.004052,0.015143,0.005685,0.000271,0.006398,0.002979,0.012890,0.018854,0.021837
2020-10-12,0.005122,0.022209,NaN,0.002614,0.007931,0.003427,0.009307,0.011767,0.006829,2.068822e-64,...,0.011373,0.003666,0.013326,0.004984,0.000135,0.005847,0.001490,0.010554,0.015937,0.023466


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-10      0.008232  0.003362  0.005272  0.011067  0.006693  0.002586   
2020-10-11      0.005960  0.002731  0.002636  0.012420  0.005987  0.002096   
2020-10-12      0.004555  0.002736  0.002380  0.007569  0.005621  0.001849   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-10      0.008011  0.006973  0.006247  0.007588  ...  0.029182   
2020-10-11      0.007157  0.005841  0.003124  0.006797  ...  0.040398   
2020-10-12      0.005666  0.005524  0.002770  0.006384  ...  0.040137   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-10      0.026837  0.036695  0.009002   0.014548  0.009260  0.006893   
2020-10-11      0.039281  0.028348  0.012135   0.004749  0.007670  0.004766   
2020-10-12      0.052427  0.038684  0.006067   0.002374  0.013654  0.003699   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-10      -0.999999  0.007869  0.062272  
2020-10-11      -0.999999  0.007967  0.031136  
2020-10-12      -0.999999  0.007983  0.015568  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-10,0.006114,0.003289,NaN,0.001757,0.014901,0.004682,0.004045,0.000361,0.002298,NaN,...,0.011308,0.005321,0.009869,2.396543e-22,0.000006,0.004088,0.001725,0.012176,0.011141,0.002417
2020-10-11,0.003057,0.001645,NaN,0.000878,0.012928,0.002703,0.002259,0.000181,0.001915,NaN,...,0.007286,0.003487,0.011797,1.198272e-22,0.000003,0.002641,0.000863,0.003504,0.007971,0.001209
2020-10-12,0.001716,0.000822,NaN,0.000439,0.011881,0.001563,0.001840,0.000311,0.002486,NaN,...,0.004908,0.002451,0.010734,5.991358e-23,0.000002,0.001768,0.000431,0.005648,0.007057,0.000604


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                   \
Admin2           Autauga   Baldwin   Barbour     Bibb    Blount   Bullock   
2020-10-10      0.004616  0.028024  0.035714  0.02496  0.004167  0.002061   
2020-10-11      0.002308  0.014012  0.017857  0.01248  0.002083  0.001031   
2020-10-12      0.001154  0.007006  0.008929  0.00624  0.001042  0.000515   

Province_State                                              ...       Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...          Park   
2020-10-10      0.000006  0.033100  0.001488 -2.258113e-06  ...  5.960464e-08   
2020-10-11      0.000003  0.016550  0.000744 -1.129056e-06  ...  2.980232e-08   
2020-10-12      0.000002  0.008275  0.000372 -5.645282e-07  ...  1.490116e-08   

Province_State                                                       \
Admin2         Platte      Sheridan Sublette Sweetwater Teton Uinta   
2020-10-10        0.0  8.940697e-08      0.0        0.0   0.0   0.0   
2020-10-11        0.0  4.470348e-08      0.0        0.0   0.0   0.0   
2020-10-12        0.0  2.235174e-08      0.0        0.0   0.0   0.0   

Province_State                                
Admin2            Unassigned Washakie Weston  
2020-10-10     -7.947528e-08      0.0    NaN  
2020-10-11     -3.406084e-08      0.0    NaN  
2020-10-12     -5.333333e-01      0.0    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201012,AK,10884,NaN,495043,NaN,505927,50.0,NaN,NaN,...,505927,0,0,1d12a6644f8d97132e890998a17090d44e70d4e1,0,0,0,0,0,NaN
1,20201012,AL,166076,19853.0,1068387,NaN,1214610,856.0,18179.0,NaN,...,1234463,1,0,feb6ff2653368253706dc08cae65080d91c5d680,0,0,0,0,0,NaN
2,20201012,AR,93487,4617.0,1066196,NaN,1155066,608.0,5960.0,256.0,...,1159683,17,50,a7e93a35dd45410111c241e756412d06c15cbedf,0,0,0,0,0,NaN
3,20201012,AS,0,NaN,1616,NaN,1616,NaN,NaN,NaN,...,1616,0,0,b24847bbdc4c2cd264057ba37db2a920a16d406e,0,0,0,0,0,NaN
4,20201012,AZ,226050,4911.0,1353061,NaN,1574200,667.0,20257.0,155.0,...,1579111,0,4,5b3f0b63a27f6be48971a371b3e73755943a1773,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201012,AK,10884.0,NaN,495043.0,NaN,505927.0,50.0,NaN,NaN,...,505927,0,0,1d12a6644f8d97132e890998a17090d44e70d4e1,0,0,0,0,0,NaN
1,20201012,AL,166076.0,19853.0,1068387.0,NaN,1214610.0,856.0,18179.0,NaN,...,1234463,1,0,feb6ff2653368253706dc08cae65080d91c5d680,0,0,0,0,0,NaN
2,20201012,AR,93487.0,4617.0,1066196.0,NaN,1155066.0,608.0,5960.0,256.0,...,1159683,17,50,a7e93a35dd45410111c241e756412d06c15cbedf,0,0,0,0,0,NaN
3,20201012,AS,0.0,NaN,1616.0,NaN,1616.0,NaN,NaN,NaN,...,1616,0,0,b24847bbdc4c2cd264057ba37db2a920a16d406e,0,0,0,0,0,NaN
4,20201012,AZ,226050.0,4911.0,1353061.0,NaN,1574200.0,667.0,20257.0,155.0,...,1579111,0,4,5b3f0b63a27f6be48971a371b3e73755943a1773,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-12,AK,10884,NaN,495043,NaN,505927,50.0,NaN,NaN,NaN,...,505927,0,0,1d12a6644f8d97132e890998a17090d44e70d4e1,0,0,0,0,0,NaN
2020-10-12,AL,166076,19853.0,1068387,NaN,1214610,856.0,18179.0,NaN,1892.0,...,1234463,1,0,feb6ff2653368253706dc08cae65080d91c5d680,0,0,0,0,0,NaN
2020-10-12,AR,93487,4617.0,1066196,NaN,1155066,608.0,5960.0,256.0,NaN,...,1159683,17,50,a7e93a35dd45410111c241e756412d06c15cbedf,0,0,0,0,0,NaN
2020-10-12,AS,0,NaN,1616,NaN,1616,NaN,NaN,NaN,NaN,...,1616,0,0,b24847bbdc4c2cd264057ba37db2a920a16d406e,0,0,0,0,0,NaN
2020-10-12,AZ,226050,4911.0,1353061,NaN,1574200,667.0,20257.0,155.0,NaN,...,1579111,0,4,5b3f0b63a27f6be48971a371b3e73755943a1773,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-12,AK,10884.0,NaN,495043.0,NaN,505927.0,50.0,NaN,NaN,NaN,...,505927,0,0,1d12a6644f8d97132e890998a17090d44e70d4e1,0,0,0,0,0,NaN
2020-10-12,AL,166076.0,19853.0,1068387.0,NaN,1214610.0,856.0,18179.0,NaN,1892.0,...,1234463,1,0,feb6ff2653368253706dc08cae65080d91c5d680,0,0,0,0,0,NaN
2020-10-12,AR,93487.0,4617.0,1066196.0,NaN,1155066.0,608.0,5960.0,256.0,NaN,...,1159683,17,50,a7e93a35dd45410111c241e756412d06c15cbedf,0,0,0,0,0,NaN
2020-10-12,AS,0.0,NaN,1616.0,NaN,1616.0,NaN,NaN,NaN,NaN,...,1616,0,0,b24847bbdc4c2cd264057ba37db2a920a16d406e,0,0,0,0,0,NaN
2020-10-12,AZ,226050.0,4911.0,1353061.0,NaN,1574200.0,667.0,20257.0,155.0,NaN,...,1579111,0,4,5b3f0b63a27f6be48971a371b3e73755943a1773,0,0,0,0,0,NaN
